In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker 
import geopandas as gpd
import plotly.express as px
import folium 
from sklearn.cluster import KMeans

In [ ]:
Retail = pd.read_csv('data/Online_retail.csv')
Retail.head()

### 1. Exploratory Data Analysis

In [ ]:
Retail.info()

In [ ]:
Retail.describe().transpose()

In [ ]:
Retail.shape

In [ ]:
Retail.columns

In [ ]:
Retail['InvoiceDate']= pd.to_datetime(Retail['InvoiceDate'], format= '%m/%d/%y %H:%M')

In [ ]:
##View countries in dataset
Retail['Country'].value_counts()

In [ ]:
# Change country name
Retail['Country'].replace({'EIRE': 'Ireland'}, inplace= True)

In [ ]:
## Check for missing values
Retail.isnull().sum()

In [ ]:
## Dealing with missing values 
## Data contains some missing customerid, due to information constraints, i chose to drop missing data 

Retail.dropna(subset='CustomerID',axis=0, inplace=True)


In [ ]:
Retail.isnull().sum()

In [ ]:
## Create Total purchase value 

Retail.eval('Purchase_Value = Quantity*UnitPrice', inplace=True)

In [ ]:
## Check for negative values in quantity(refund or returns)
Retail[Retail['Quantity'] < 0].head()

### 2. Global Sales Analysis

In [ ]:
def sales_breakdown():
    Total_sales = Retail.loc[Retail['Quantity'] > 0, 'Purchase_Value'].sum().round()
    Refunds_returns = Retail.loc[Retail['Quantity'] < 0, 'Purchase_Value'].sum().round()
    Total_sales_post_refund = Retail['Purchase_Value'].sum().round()
    
    print(f'Total sales: ${Total_sales}')
    print(f'Total sales after refund : ${Total_sales_post_refund}')
    print(f'Refund: ${Refunds_returns}')

In [ ]:
refunds= Retail[Retail['Quantity'] < 0]
refunds.to_csv('refunds_data.csv', index=False)

In [ ]:
Retail.to_csv('Retail_data.csv', index= False)

In [ ]:
sales_breakdown()

In [ ]:
## Sales breakdown by country, top 5 and least 5 markets. 

Total_sales = Retail.groupby('Country')['Purchase_Value'].sum().reset_index()
Top_5 = Total_sales.sort_values('Purchase_Value', ascending=False).head()
Least_5 = Total_sales.sort_values('Purchase_Value', ascending=True).head()

In [ ]:
## Visualisation top 5 countries

plt.figure(figsize=(10,6))
ax = sns.barplot(Top_5, x= 'Country', y= 'Purchase_Value')

ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

ax.grid(True, linestyle='--', alpha=0.5)

for index, value in enumerate(Top_5['Purchase_Value']):
    ax.text(index, value, f'${value:,.0f}', ha = 'center', va= 'bottom', weight= 'bold')

plt.title('Top Countries By Sales', weight='bold')
plt.xlabel('Countries')
plt.ylabel('Sales value')
plt.show()

In [ ]:
## Visualisation Least 5 

plt.figure(figsize=(10,6))
ax1 = sns.barplot(Least_5, x= 'Country', y= 'Purchase_Value')

ax1.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

ax1.grid(True, linestyle='--', alpha=0.5)

for index, value in enumerate(Least_5['Purchase_Value']):
    ax1.text(index, value, f'${value:,.0f}', ha = 'center', va= 'bottom', weight= 'bold')

plt.title('Least Performing Countries By Sales', weight='bold')
plt.xlabel('Countries')
plt.ylabel('Sales value')
plt.show()

In [ ]:
## Sales by Quarter
Quartely_sales = Retail.groupby(Retail['InvoiceDate'].dt.to_period('Q')).sum('Purchase_Value').reset_index()
Quartely_sales['InvoiceDate'] = Quartely_sales['InvoiceDate'].astype(str)

##Visualisation
plt.figure(figsize=(10,6))
ax2 = sns.lineplot(Quartely_sales, x= 'InvoiceDate',y = 'Purchase_Value', color ='red')
ax2.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

for index, value in enumerate(Quartely_sales['Purchase_Value']):
    ax2.text(index,value, f'${value:,.0f}', ha='center', va ='bottom', weight ='bold', color ='blue')
                              

ax2.grid(True, linestyle='--', alpha=0.5)        
plt.title('Quartely Performance', weight='bold')
plt.xlabel('Quarters')
plt.ylabel('Sales value')
plt.show()

In [ ]:
## Disaggregating further to see performance of top 5 excluding United Kingdom by quarter

Retail['Quarter'] = Retail['InvoiceDate'].dt.to_period('Q')

# Group by Country and Quarter, summing up Purchase_Value
Quarterly_sales_by_country = Retail.groupby(['Country', 'Quarter']).sum('Purchase_Value').reset_index()
Quarterly_sales_by_country['Quarter'] = Quarterly_sales_by_country['Quarter'].astype(str)


Quarter_exc_Uk= Quarterly_sales_by_country[Quarterly_sales_by_country['Country'] != 'United Kingdom']
Quarter_exc_Uk = Quarter_exc_Uk[Quarter_exc_Uk['Country'].isin(['Netherland','Australia','Ireland','Germany','France'])]
Quarter_exc_Uk


plt.figure(figsize=(10,6))
ax3= sns.lineplot(Quarter_exc_Uk, x='Quarter', y='Purchase_Value', hue= 'Country')

ax3.grid(True, linestyle='--', alpha=0.5)
ax3.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

ax.grid(True, linestyle='--', alpha=0.5)        
plt.title('Quartely Performance', weight='bold')
plt.xlabel('Quarters')
plt.ylabel('Sales value')
plt.show()

In [ ]:
#Top selling Products by Value
Top_sellers = Retail.groupby('Description')['Purchase_Value'].sum().sort_values(ascending=False).head()

# Top selling products by Quantity
Top_quantity = Retail.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head()


In [ ]:
# Load a GeoJSON file for world map (replace 'path/to/world.geojson' with your GeoJSON file)
world_geojson = 'countries.geo.json'

# Create a Folium map centered on the world
m = folium.Map(location=[0, 0], zoom_start=2)

# Add choropleth layer
folium.Choropleth(
    geo_data=world_geojson,
    data=rr,
    columns=['Country', 'Purchase_Value'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Purchase Value'
).add_to(m)

# Display the map
m

In [ ]:
# Plot the distribution using Seaborn
plt.figure(figsize=(8, 6))
sns.displot(Customer['Purchase_Value'], bins=10, kde=True, color='#3498db')
plt.title('Distribution of Spending Score')
plt.xlabel('Spending Score')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### 3. Customer Analysis: Creating Customer Segmentation Using Clusters


In [ ]:
## Finding customer spend
Customer = Retail.groupby(['CustomerID','Country'])[['Purchase_Value','Quantity']].sum().round(2).reset_index()

#Top 5 customers by the value of their purchase
Top_5_Customer = Customer.sort_values('Purchase_Value', ascending=False).head(5)

##Top Customers in each country in our dataset
Top_cus_countries= Customer.groupby('Country').apply(lambda group: group.nlargest(1, 'Purchase_Value')).reset_index(drop=True)


In [ ]:
## To create the customer segmentation, we will apply a K-means model
x = Customer[['Purchase_Value','Quantity']]


In [ ]:
fig=plt.figure(figsize=(10,6))
ax=sns.scatterplot(x=Customer["Quantity"],y=Customer["Purchase_Value"],color="#b20710")

fig.patch.set_facecolor('#f6f5f5')

ax.set_xlim(10000, 80000)

ax.set_facecolor('#f5f6f6')
for loc in ['right', 'top']:
    ax.spines[loc].set_visible(False)
 
fig.text(0.2,1,"Plotting Quantity Against Purchase Spending",**{'font':'serif', 'size':18,'weight':'bold'}, alpha = 1)
fig.text(0.1,0.90,"It seems like there are already some clusters :",**{'font':'serif', 'size':14,}, alpha = 1)


plt.show()

In [ ]:
## Selecting the optimal number of clusters 
wcss = []
for i in range(1, 11):
    km = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    km.fit(x)
    wcss.append(km.inertia_)
fig=plt.figure(figsize=(10,6))  
fig.patch.set_facecolor('#f6f5f5')

plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method', fontsize = 20)
plt.xlabel('No. of Clusters')
plt.ylabel('wcss')
fig.text(0.5,0.4,"The best k-value is 5")
plt.show()

In [ ]:
## Optimal number of Clusters is 5, we create the clusters.
km = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
y_means = km.fit_predict(x)

In [ ]:
##Creating a data frame that contains the different clusters 
cluster = pd.Series(y_means, name='Cluster')
clustered_cus = pd.concat([Customer, cluster], axis=1)

In [ ]:
## Changing the Tiers of the cluster to better represent them
# Diamond,Platinum,Gold,Silver, Bronze
clustered_cus['Cluster'].replace({0:'Bronze', 1:'Diamond', 2:'Platinum', 3:'Gold',
                                                           4:'Silver'}, inplace=True)


In [ ]:
plt.figure(figsize=(10,6))
ax5 = sns.countplot(clustered_cus, y='Cluster')
ax5.grid(True, linestyle='--', alpha=0.5)

plt.title('Number of customers in each class')
print('Total number of customers in each cluster:',clustered_cus['Cluster'].value_counts())